# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 30 2023 9:32AM,256154,16,EMERSA-9093859,"Emersa Waterbox, LLC",Released
1,Jan 30 2023 9:31AM,256153,10,CTF0011623,"Citieffe, Inc.",Released
2,Jan 30 2023 9:28AM,256152,16,EMERSA-9093858,"Emersa Waterbox, LLC",Released
3,Jan 30 2023 9:21AM,256151,16,EMERSA-9093855,"Emersa Waterbox, LLC",Released
4,Jan 30 2023 9:17AM,256150,19,9093850,"Snap Medical Industries, LLC",Released
5,Jan 30 2023 9:17AM,256150,19,9093852,"Snap Medical Industries, LLC",Released
6,Jan 30 2023 9:16AM,256147,10,0086339452,ISDIN Corporation,Released
7,Jan 30 2023 9:16AM,256147,10,0086339453,ISDIN Corporation,Released
8,Jan 30 2023 9:16AM,256147,10,0086339455,ISDIN Corporation,Released
9,Jan 30 2023 9:16AM,256147,10,0086339463,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
8,256150,Released,2
9,256151,Released,1
10,256152,Released,1
11,256153,Released,1
12,256154,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
256150,NaN,2.0
256151,NaN,1.0
256152,NaN,1.0
256153,NaN,1.0
256154,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
256139,0.0,13.0
256140,0.0,1.0
256141,3.0,4.0
256142,0.0,1.0
256145,0.0,28.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
256139,0,13
256140,0,1
256141,3,4
256142,0,1
256145,0,28


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,256139,0,13
1,256140,0,1
2,256141,3,4
3,256142,0,1
4,256145,0,28


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,256139,,13
1,256140,,1
2,256141,3,4
3,256142,,1
4,256145,,28


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 30 2023 9:32AM,256154,16,"Emersa Waterbox, LLC"
1,Jan 30 2023 9:31AM,256153,10,"Citieffe, Inc."
2,Jan 30 2023 9:28AM,256152,16,"Emersa Waterbox, LLC"
3,Jan 30 2023 9:21AM,256151,16,"Emersa Waterbox, LLC"
4,Jan 30 2023 9:17AM,256150,19,"Snap Medical Industries, LLC"
6,Jan 30 2023 9:16AM,256147,10,ISDIN Corporation
37,Jan 30 2023 9:12AM,256146,10,ISDIN Corporation
80,Jan 30 2023 9:09AM,256145,10,"Digital Brands, LLC"
108,Jan 30 2023 8:56AM,256142,10,Bio-PRF
109,Jan 30 2023 8:54AM,256141,10,Bio-PRF


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jan 30 2023 9:32AM,256154,16,"Emersa Waterbox, LLC",,1
1,Jan 30 2023 9:31AM,256153,10,"Citieffe, Inc.",,1
2,Jan 30 2023 9:28AM,256152,16,"Emersa Waterbox, LLC",,1
3,Jan 30 2023 9:21AM,256151,16,"Emersa Waterbox, LLC",,1
4,Jan 30 2023 9:17AM,256150,19,"Snap Medical Industries, LLC",,2
5,Jan 30 2023 9:16AM,256147,10,ISDIN Corporation,,31
6,Jan 30 2023 9:12AM,256146,10,ISDIN Corporation,,43
7,Jan 30 2023 9:09AM,256145,10,"Digital Brands, LLC",,28
8,Jan 30 2023 8:56AM,256142,10,Bio-PRF,,1
9,Jan 30 2023 8:54AM,256141,10,Bio-PRF,3,4


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 30 2023 9:32AM,256154,16,"Emersa Waterbox, LLC",1,
1,Jan 30 2023 9:31AM,256153,10,"Citieffe, Inc.",1,
2,Jan 30 2023 9:28AM,256152,16,"Emersa Waterbox, LLC",1,
3,Jan 30 2023 9:21AM,256151,16,"Emersa Waterbox, LLC",1,
4,Jan 30 2023 9:17AM,256150,19,"Snap Medical Industries, LLC",2,
5,Jan 30 2023 9:16AM,256147,10,ISDIN Corporation,31,
6,Jan 30 2023 9:12AM,256146,10,ISDIN Corporation,43,
7,Jan 30 2023 9:09AM,256145,10,"Digital Brands, LLC",28,
8,Jan 30 2023 8:56AM,256142,10,Bio-PRF,1,
9,Jan 30 2023 8:54AM,256141,10,Bio-PRF,4,3


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 30 2023 9:32AM,256154,16,"Emersa Waterbox, LLC",1,
1,Jan 30 2023 9:31AM,256153,10,"Citieffe, Inc.",1,
2,Jan 30 2023 9:28AM,256152,16,"Emersa Waterbox, LLC",1,
3,Jan 30 2023 9:21AM,256151,16,"Emersa Waterbox, LLC",1,
4,Jan 30 2023 9:17AM,256150,19,"Snap Medical Industries, LLC",2,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jan 30 2023 9:32AM,256154,16,"Emersa Waterbox, LLC",1,NaN
1,Jan 30 2023 9:31AM,256153,10,"Citieffe, Inc.",1,NaN
2,Jan 30 2023 9:28AM,256152,16,"Emersa Waterbox, LLC",1,NaN
3,Jan 30 2023 9:21AM,256151,16,"Emersa Waterbox, LLC",1,NaN
4,Jan 30 2023 9:17AM,256150,19,"Snap Medical Industries, LLC",2,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 30 2023 9:32AM,256154,16,"Emersa Waterbox, LLC",1,0.0
1,Jan 30 2023 9:31AM,256153,10,"Citieffe, Inc.",1,0.0
2,Jan 30 2023 9:28AM,256152,16,"Emersa Waterbox, LLC",1,0.0
3,Jan 30 2023 9:21AM,256151,16,"Emersa Waterbox, LLC",1,0.0
4,Jan 30 2023 9:17AM,256150,19,"Snap Medical Industries, LLC",2,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1793013,121,3.0
12,256140,1,0.0
16,768457,3,0.0
19,256150,2,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1793013,121,3.0
1,12,256140,1,0.0
2,16,768457,3,0.0
3,19,256150,2,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,121,3.0
1,12,1,0.0
2,16,3,0.0
3,19,2,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,121.0
1,12,Released,1.0
2,16,Released,3.0
3,19,Released,2.0
4,10,Executing,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19
Status,,,,
Executing,3.0,0.0,0.0,0.0
Released,121.0,1.0,3.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19
0,Executing,3.0,0.0,0.0,0.0
1,Released,121.0,1.0,3.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19
0,Executing,3.0,0.0,0.0,0.0
1,Released,121.0,1.0,3.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()